In [4]:
import pandas as pd
import numpy as np
from transformers import BertTokenizer
import ast
import torch
import pickle as pkl

ingredients_db = pkl.load(open('../Dataset/calories_embedded.pkl', 'rb'))

In [8]:
print(ingredients_db['Cals_per100grams'][0][:-4])

62


In [2]:
temp_mp = {'Simple White Cake': [[280, 'all purpose flour'], [200, 'granulated sugar'], [100, 'unsalted butter'], [120, 'large eggs'], [10, 'vanilla extract']], 'Best Soft Chocolate Chip Cookies': [[225, 'all purpose flour'], [100, 'white sugar'], [50, 'light brown sugar'], [115, 'salted butter'], [60, 'large egg'], [40, 'vanilla'], [115, 'chocolate chips']], 'Best Homemade Brownies': [[190, 'all purpose flour'], [225, 'granulated sugar'], [80, 'cocoa powder'], [60, 'powdered sugar'], [115, 'dark chocolate chips'], [120, 'large eggs'], [60, 'canola oil'], [15, 'vanilla extract']]}

In [8]:
def cosine(a,b):
    print(type(a))
    print(type(b))
    # if a == 0 or b == 0:
    #     return 0
    return np.dot(a,b)/(np.linalg.norm(a)*np.linalg.norm(b))

In [ ]:
def check_meal_plan():
    print("---CREATE MEAL PLAN---")
    meal_plan = temp_mp
    current_calories = 0
    calorie_goal = 2000

    # encoder
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

    for dish in meal_plan:
        for grams, ingredient in meal_plan[dish]:
            # vectorize ingredient
            print(ingredient)
            encoded_input = tokenizer(ingredient, max_length = 50, truncation=False, return_tensors='pt')
            print(encoded_input)
            encoded_input = pd.Series(encoded_input)
            # compare to db
            most_similar = (0,"")
            for index,item in enumerate(ingredients_db['embedding']):
                if cosine(item,encoded_input) > most_similar[0]:
                    most_similar = (cosine(item,encoded_input),index)

            # calculate calories
            current_calories += grams * ingredients_db['Data.Carbohydrate'][most_similar[1]]

    if current_calories > calorie_goal:
        return {"redo": True}
    else: 
        return {"redo": False, "current_calories": current_calories} 

check_meal_plan()

---CREATE MEAL PLAN---
all purpose flour
{'input_ids': tensor([[  101,  2035,  3800, 13724,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}
<class 'numpy.ndarray'>
<class 'pandas.core.series.Series'>


ValueError: shapes (1,768) and (3,) not aligned: 768 (dim 1) != 3 (dim 0)